# Setup

In [605]:
# Import necessary packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re
import math

# Import Raw Data

## Reference
https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html

In [606]:
# Import General Metrics data, and take a peek
dfRaw_generalMetrics = pd.read_excel('data/raw_data.xlsx', sheet_name='General Metrics');
dfRaw_generalMetrics

,Overall Rating,Total Number of Reviews,Webscraping Datetime
0,3.9,329,2023-01-29 20:55:10


In [607]:
# Import Stars Distribution data, and take a peek
dfRaw_starsDistribution = pd.read_excel('data/raw_data.xlsx', sheet_name='Stars Distribution');
dfRaw_starsDistribution


,Stars Given,Number of Reviews
0,5,182
1,4,53
2,3,21
3,2,15
4,1,58


In [608]:
# Import Reviews data, and take a peek
dfRaw_reviews = pd.read_excel('data/raw_data.xlsx', sheet_name='Reviews');
print(dfRaw_reviews.info());
dfRaw_reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name                 327 non-null    object
 1   Total Reviews Given  327 non-null    object
 2   Time of Review       327 non-null    object
 3   Review               230 non-null    object
 4   Stars Given          322 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB
None


,Name,Total Reviews Given,Time of Review,Review,Stars Given
0,Vielka Duran,Local Guide · 41 reviews,6 hours ago,NaN,5 stars
1,LASHLOVE33,3 reviews,2 weeks ago,Hit and miss. Staff are either extremely rude ...,2 stars
2,Linda Wilke,4 reviews,4 weeks ago,Tried calling this boutique to ask some questi...,2 stars
3,Alex Reyes,2 reviews,a month ago,Response from the owner a month ago,3 stars
4,Hugo Nakamura,Local Guide · 25 reviews,a month ago,"EDIT: The second time coming to the store, the...",5 stars


# Clean & Transform the Raw Reviews Data

## Reference
https://www.learndatasci.com/solutions/python-string-contains/#:~:text=The%20easiest%20and%20most%20effective,can't%20find%20the%20substring.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html

https://www.digitalocean.com/community/tutorials/pandas-dataframe-apply-examples

https://stackoverflow.com/questions/15891038/change-column-type-in-pandas

https://www.digitalocean.com/community/tutorials/python-remove-character-from-string

https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-table-style/

https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

https://statisticsglobe.com/add-subtract-weeks-from-date-python

https://www.geeksforgeeks.org/how-to-add-and-subtract-days-using-datetime-in-python/

https://stackoverflow.com/questions/546321/how-do-i-calculate-the-date-six-months-from-the-current-date-using-the-datetime

https://rollbar.com/blog/python-typeerror-nonetype-object-is-not-iterable/

In [609]:
# Create a copy of raw Reviews data for data cleaning
dfCleaned_reviews = dfRaw_reviews.copy();

# Change name of 'Name' column to 'Reviewer Name'
dfCleaned_reviews = dfCleaned_reviews.rename(columns={'Name':'Reviewer Name'});
dfCleaned_reviews.columns

Index(['Reviewer Name', 'Total Reviews Given', 'Time of Review', 'Review',
       'Stars Given'],
      dtype='object')

In [610]:
# Retrieve reviewer title from 'Total Reviews Given' column 
# and create a new 'Reviewer Title' column
def get_reviewerTitle(df):
    df['Reviewer Title'] = None;
    for i in df.index:
        if '·' in df.loc[i, 'Total Reviews Given']:
            colValue = df.loc[i, 'Total Reviews Given'].split('·');
            df.loc[i, 'Reviewer Title'] = colValue[0].strip();
            df.loc[i, 'Total Reviews Given'] = colValue[1].strip();
    return df;

dfCleaned_reviews = get_reviewerTitle(dfCleaned_reviews);
print(dfCleaned_reviews.columns);
dfCleaned_reviews.head(1)


Index(['Reviewer Name', 'Total Reviews Given', 'Time of Review', 'Review',
       'Stars Given', 'Reviewer Title'],
      dtype='object')


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
0,Vielka Duran,41 reviews,6 hours ago,NaN,5 stars,Local Guide


In [611]:
# Clean 'Total Reviews Given' column
dfCleaned_reviews['Total Reviews Given'] = dfCleaned_reviews['Total Reviews Given'].apply(lambda x: x.split(" ")[0]);
dfCleaned_reviews.head(1)


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
0,Vielka Duran,41,6 hours ago,NaN,5 stars,Local Guide


In [612]:
# Clean the 'Total Reviews Given' column
def convert_totalReviewsGivenToNumeric(df):
    # Parse value in 'Total Reviews Given' column to numeric
    # and record which rows are unable to parse to numeric
    index_list = [];
    for i in df.index:
        try:
            df.loc[i, 'Total Reviews Given'] = int(df.loc[i, 'Total Reviews Given']);
        except ValueError:
            print(f"Unable to parse to integer the number of reviews given by {df.loc[i, 'Reviewer Name']}");
            index_list.append(i);
    # From the rows where the values in the 'Total Reviews Given' column 
    # could not be parsed to numeric, remove any instance of ","
    # or equate the value to 0
    for i in index_list:
        if ',' in df.loc[i, 'Total Reviews Given']:
            df.loc[i, 'Total Reviews Given'] = df.loc[i, 'Total Reviews Given'].replace(',','');
        else:
            df.loc[i, 'Total Reviews Given'] = 0;
    df['Total Reviews Given'] = pd.to_numeric(df['Total Reviews Given']);
    return df;

dfCleaned_reviews = convert_totalReviewsGivenToNumeric(dfCleaned_reviews);
print(dfCleaned_reviews['Total Reviews Given'].dtype);
dfCleaned_reviews.head(1)


Unable to parse to integer the number of reviews given by MiHyun Kang
Unable to parse to integer the number of reviews given by Alex Passini
Unable to parse to integer the number of reviews given by Armando yaya
int64


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
0,Vielka Duran,41,6 hours ago,NaN,5 stars,Local Guide


In [613]:
# Check the rows where the 'Total Reviews Given' column value could not 
# initially be parsed to numeric had appropriate adjustments so that they later were
# parsed correctly

display(dfCleaned_reviews[dfCleaned_reviews['Reviewer Name'] == 'MiHyun Kang'])

display(dfCleaned_reviews[dfCleaned_reviews['Reviewer Name'] == 'Alex Passini'])

display(dfCleaned_reviews[dfCleaned_reviews['Reviewer Name'] == 'Armando yaya'])

,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
233,MiHyun Kang,0,3 years ago,NaN,4 stars,None


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
296,Alex Passini,2441,4 years ago,Very friendly staff and a great selection of c...,5 stars,Local Guide


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
314,Armando yaya,1090,4 years ago,love the coffee,5 stars,Local Guide


In [614]:
# Check the unique values in 'Time of Review' column
dfCleaned_reviews['Time of Review'].unique()

array(['6 hours ago', '2 weeks ago', '4 weeks ago', 'a month ago',
       '2 months ago', '3 months ago', '4 months ago', '5 months ago',
       '6 months ago', '7 months ago', '8 months ago', '9 months ago',
       '11 months ago', 'a year ago', '2 years ago', ' 5 stars ',
       '3 years ago', '4 years ago', ' 1 star '], dtype=object)

In [615]:
# In the above code output, it is evident that there has been some web scraping error
# for the following rows, which require some adjustment such that
# the value for 'Total Reviews Given' is moved to 'Time of Review' column
# and the value for 'Time of Review' is moved to 'Stars Given'

display(dfCleaned_reviews[dfCleaned_reviews['Time of Review'] == ' 5 stars '])

display(dfCleaned_reviews[dfCleaned_reviews['Time of Review'] == ' 1 star '])

,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
100,Jason Wu,2,5 stars,NaN,NaN,None
108,Randy Rhoads,2,5 stars,NaN,NaN,None
171,박유경,3,5 stars,NaN,NaN,None
192,kenny Chow,3,5 stars,NaN,NaN,None


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
323,Jina Lee,4,1 star,NaN,NaN,None


In [616]:
# Get index of rows where web scraped data for 'Total Reviews Given' 
# and 'Time of Review' is invalid
def get_rowindexWithWrongValues(df, timeOfReview_invalidValue):
    index_list = [];
    for value in timeOfReview_invalidValue:
        for index in df[df['Time of Review'] == value].index.tolist():
            index_list.append(index);
    return index_list;

invalidWebScrape_index_list = get_rowindexWithWrongValues(
    df=dfCleaned_reviews, 
    timeOfReview_invalidValue=[' 5 stars ', ' 1 star ']
);
invalidWebScrape_index_list


[100, 108, 171, 192, 323]

In [617]:
# Shuffle 'Total Reviews Given' & 'Time of Review' column values for
# rows with above indices
def shuffle_totalReviewsGiven_timeOfReview_columnValues(df, index_list):
    for index in index_list:
        df.loc[index, 'Stars Given'] = df.loc[index, 'Time of Review'];
        df.loc[index, 'Time of Review'] = str(int(df.loc[index, 'Total Reviews Given'])) + " years ago";
        df.loc[index, 'Total Reviews Given'] = 0;
    return df;

dfCleaned_reviews = shuffle_totalReviewsGiven_timeOfReview_columnValues(
    df=dfCleaned_reviews, 
    index_list=invalidWebScrape_index_list);

print(dfCleaned_reviews['Time of Review'].unique());

dfCleaned_reviews.loc[invalidWebScrape_index_list]


['6 hours ago' '2 weeks ago' '4 weeks ago' 'a month ago' '2 months ago'
 '3 months ago' '4 months ago' '5 months ago' '6 months ago'
 '7 months ago' '8 months ago' '9 months ago' '11 months ago' 'a year ago'
 '2 years ago' '3 years ago' '4 years ago']


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
100,Jason Wu,0,2 years ago,NaN,5 stars,None
108,Randy Rhoads,0,2 years ago,NaN,5 stars,None
171,박유경,0,3 years ago,NaN,5 stars,None
192,kenny Chow,0,3 years ago,NaN,5 stars,None
323,Jina Lee,0,4 years ago,NaN,1 star,None


In [618]:
# Find the year of review using the 'Webscraping Datetime' value from general metrics data 
# and 'Time of Review' column values from reviews data
webscrapeDatetime = dfRaw_generalMetrics['Webscraping Datetime'].values.tolist()[0];
print(f'Webscraping Datetime: {str(webscrapeDatetime)}');

def get_yearOfReview(df, webscrapeDatetime):
    df['Webscraping Datetime'] = pd.to_datetime(webscrapeDatetime);
    df['Year of Review'] = 0;
    
    for i in df.index:
        if df.loc[i, 'Time of Review'].split(" ")[0] == 'a':
            duration = 1;
        else:
            duration = int(df.loc[i, 'Time of Review'].split(" ")[0]);
        
        if 'hour' in df.loc[i, 'Time of Review']:
            timeDeduction = timedelta(hours=duration);
        elif 'week' in df.loc[i, 'Time of Review']:
            timeDeduction = timedelta(weeks=duration);
        elif 'month' in df.loc[i, 'Time of Review']:
            timeDeduction = relativedelta(months=duration);
        elif 'year' in df.loc[i, 'Time of Review']:
            timeDeduction = relativedelta(years=duration);
        
        yearOfReview = pd.to_datetime(webscrapeDatetime) - timeDeduction;
        df.loc[i, 'Year of Review'] = yearOfReview.year;
    return df;

dfCleaned_reviews = get_yearOfReview(dfCleaned_reviews, webscrapeDatetime);
dfCleaned_reviews.head(1)

Webscraping Datetime: 2023-01-29 20:55:10


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title,Webscraping Datetime,Year of Review
0,Vielka Duran,41,6 hours ago,NaN,5 stars,Local Guide,2023-01-29 20:55:10,2023


In [619]:
# Clean values in 'Review' column such that values showing "Response from the owner"
# and null values are accomodated for
def clean_reviews(df):
    for i in df[df['Review'].isnull()].index.tolist():
        df.loc[i, 'Review'] = None;
    
    for i in df.index:
        if df.loc[i, 'Review'] is not None:
            if 'Response from the owner' in df.loc[i, 'Review']:
                df.loc[i, 'Review'] = None;
    
    return df;

dfCleaned_reviews = clean_reviews(dfCleaned_reviews);
dfCleaned_reviews.head(1)


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title,Webscraping Datetime,Year of Review
0,Vielka Duran,41,6 hours ago,None,5 stars,Local Guide,2023-01-29 20:55:10,2023


In [620]:
# Clean 'Stars Given' column and change data type of column to numeric
dfCleaned_reviews['Stars Given'] = dfCleaned_reviews['Stars Given'].apply(lambda x: x.strip().split(" ")[0]);
dfCleaned_reviews['Stars Given'] = pd.to_numeric(dfCleaned_reviews['Stars Given']);
print(dfCleaned_reviews['Stars Given'].dtype);
dfCleaned_reviews.head(1)


int64


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title,Webscraping Datetime,Year of Review
0,Vielka Duran,41,6 hours ago,None,5,Local Guide,2023-01-29 20:55:10,2023


In [621]:
# Reorder columns of reviews dataframe
dfCleaned_reviews = dfCleaned_reviews[[
    'Reviewer Name',
    'Reviewer Title',
    'Total Reviews Given',
    'Review',
    'Stars Given',
    'Year of Review',
    'Time of Review',
    'Webscraping Datetime'
]];

print(dfCleaned_reviews.info());
dfCleaned_reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Reviewer Name         327 non-null    object        
 1   Reviewer Title        203 non-null    object        
 2   Total Reviews Given   327 non-null    int64         
 3   Review                225 non-null    object        
 4   Stars Given           327 non-null    int64         
 5   Year of Review        327 non-null    int64         
 6   Time of Review        327 non-null    object        
 7   Webscraping Datetime  327 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 20.6+ KB
None


,Reviewer Name,Reviewer Title,Total Reviews Given,Review,Stars Given,Year of Review,Time of Review,Webscraping Datetime
0,Vielka Duran,Local Guide,41,None,5,2023,6 hours ago,2023-01-29 20:55:10
1,LASHLOVE33,None,3,Hit and miss. Staff are either extremely rude ...,2,2023,2 weeks ago,2023-01-29 20:55:10
2,Linda Wilke,None,4,Tried calling this boutique to ask some questi...,2,2023,4 weeks ago,2023-01-29 20:55:10
3,Alex Reyes,None,2,None,3,2022,a month ago,2023-01-29 20:55:10
4,Hugo Nakamura,Local Guide,25,"EDIT: The second time coming to the store, the...",5,2022,a month ago,2023-01-29 20:55:10


# Export Cleaned & Transformed Reviews Data

In [623]:
dfCleaned_reviews.to_csv('data/cleanedData_Reviews.csv', index=False);